<a href="https://colab.research.google.com/github/fabiomoschella/Coursera_Capstone/blob/master/FM_Capstone_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Segmenting and Clustering Neighborhoods in Toronto¶
###PART 1:

In [0]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy  as np
import requests
import folium
import requests
import folium
import matplotlib.cm as cm
from matplotlib import  pyplot as plt
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

I use only pandas method  for extract the table  and an old wikipedia link with correct table 


In [0]:
HtmlWeb='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'
df = pd.read_html(HtmlWeb, header = 0)[0]
df.rename(columns={'Postcode':'Postalcode'},inplace=True)

We process only Postcode with borough assigned,skipped everything else.


In [21]:
df = df[df.Borough != 'Not assigned']
df.head()


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [22]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. These multiple rows will be combined into one row with the neighborhoods separated with a comma.



In [23]:
df = df.groupby(['Borough', 'Postalcode'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()


,Borough,Postalcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


Totale records


In [24]:
df.shape

(103, 3)

###PART 2:Use the Geocoder package



Use the Geocoder package to get the latitude and the longitude coordinates for all the neighborhoods in our dataframe. 



In [25]:
!pip install  geocoder 

import geocoder
import pandas as pd
import numpy as np

Read the file csv and rename the column 'Postal Code'  for the join subsequently.

In [26]:
csvfile='http://cocl.us/Geospatial_data'
dfGeo = pd.read_csv(csvfile)
dfGeo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
dfGeo

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Simple Join Dataframe between Borough and Geo information

In [27]:
sKey='Postalcode'
#BEGIN   
if sKey in df.columns: 
  df.set_index(sKey,inplace=True)
   
if sKey in dfGeo.columns: 
   dfGeo.set_index(sKey,inplace=True)
#END 

dfToronto= df.join(dfGeo,on='Postalcode',rsuffix='right_')

df.reset_index(inplace=True)
dfGeo.reset_index(inplace=True)
# not reset index  because we must use at the end of the notebook
#dfToronto.reset_index(inplace=True) 

dfToronto

,Borough,Neighbourhood,Latitude,Longitude
Postalcode,,,,
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
M4P,Central Toronto,Davisville North,43.712751,-79.390197
M4R,Central Toronto,North Toronto West,43.715383,-79.405678
M4S,Central Toronto,Davisville,43.704324,-79.388790
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...
M6C,York,Humewood-Cedarvale,43.693781,-79.428191
M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013


### Part 3: Folium Map

Explore and cluster the neighborhoods in Toronto.


In [28]:
#!pip install folium
#!pip install  geopy 
from geopy.geocoders import Nominatim 

#with this library I take the latitude and longitude coordinates
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="FM_Capstone_Project_Notebook")
location = geolocator.geocode(address)
print('The geograpical coordinate of ',address,' are {}, {}.'.format(location.latitude, location.longitude))

The geograpical coordinate of  Toronto,ON  are 43.6534817, -79.3839347.


In [29]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

Now we can explore the useful information of foursquare.


In [0]:
#add the credential information 
CLIENT_ID = 'F4FVI5L4NGVZVPRIVY0EGEZ14YL1X11PGI43FUYMVNVJJDNM'
CLIENT_SECRET = 'NEFKIQAODHFFCFHTAE3F05UE1BOJ4KJLOOPDVJE1KQHOGNJ0'


In [0]:
# create URL
VERSION = '20180605'
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    location.latitude, 
    location.longitude, 
    radius, 
    LIMIT)

In [0]:
httpResult = requests.get(url).json()
#httpResult

Now we must use a function from lab for extract the category

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
        

And now a function for explore Near by Venues with a radius of 500

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, verbose=False):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        if verbose:
            print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
venues = httpResult['response']['groups'][0]['items']
    
nvenues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nvenues =nvenues.loc[:, filtered_columns]

# extract the category for each row
nvenues['venue.categories'] = nvenues.apply(get_category_type, axis=1)

# delete from column name "venue."
nvenues.columns = [col.split(".")[-1] for col in nvenues.columns]

nvenues.head()

KeyError: ignored

We get the venue  (with Verbose=True  print all the content)




In [0]:
dfTorontoVenues = getNearbyVenues(names=dfToronto['Neighbourhood'],
                                 latitudes=dfToronto['Latitude'],
                                 longitudes=dfToronto['Longitude'],
                                 verbose=False
                                  )
dfTorontoVenues

Check all the category  and organized in pivot style.






In [0]:
dfTorontoCategory = pd.get_dummies(dfTorontoVenues[['Venue Category']], prefix="", prefix_sep="")

dfTorontoCategory['Neighborhood'] = dfTorontoVenues['Neighborhood'] 

fixed_columns = [dfTorontoCategory.columns[-1]] + list(dfTorontoCategory.columns[:-1])
dfTorontoCategory = dfTorontoCategory[fixed_columns]

dfTorontoCategory.head()

In [0]:
print('There are {} categories'.format(len(dfTorontoCategory.columns)))

Now for exploratory analysys  filter the Venues Dataframe for extract only Restaurant.

In [0]:
from google.colab import files
#dfTorontoVenues.to_excel("TorontoRestaurant.xls")
#files.download("TorontoRestaurant.xls")

dfTorontoRestaurant = dfTorontoVenues[dfTorontoVenues['Venue Category'].str.contains('Restaurant')]
dfTorontoRestaurant.head(10)

#dfTorontoRestaurant.to_excel('TorontoRestaurant.xls')
#files.download('TorontoRestaurant.xls') 


Now we Map all the Restaurant with folium

In [0]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
restaurant = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# add markers to map
for lat, lng, venue, category in zip(dfTorontoRestaurant['Venue Latitude'], dfTorontoRestaurant['Venue Longitude'], 
                                     dfTorontoRestaurant['Venue'], dfTorontoRestaurant['Venue Category']):
    #print(venue)
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='black',
        fill_opacity=0.9,
        parse_html=False).add_to(restaurant)  

restaurant


Now for exploratory analysis we extract only Italian restaurant


In [0]:
dfTIRestaurant = dfTorontoVenues[dfTorontoVenues['Venue Category']=="Italian Restaurant"]
dfTIRestaurant.head(10)

Now we Map all the Italian Restaurant with folium


In [0]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
italianRestaurant = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# add markers to map
for lat, lng, venue, category in zip(dfTIRestaurant['Venue Latitude'], dfTIRestaurant['Venue Longitude'], dfTIRestaurant['Venue'], dfTIRestaurant['Venue Category']):
    #print(venue)
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(italianRestaurant)  

italianRestaurant



NeighBorough and Italian Restaurant

In [0]:
# add markers to map
for lat, lng, venue, category in zip(dfTIRestaurant['Venue Latitude'], dfTIRestaurant['Venue Longitude'], dfTIRestaurant['Venue'], dfTIRestaurant['Venue Category']):
    #print(venue)
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto 


Now we group rows by neighbourhood and extract mean

In [0]:
dfTCategoryGroup  = dfTorontoCategory.groupby('Neighborhood').mean().reset_index()
dfTCategoryGroup.head()

Relationship between neighborhood and Italian Restaurant </h3>


In [0]:

#TEST GROUP BY
#toronto_part = dfTorontoCategory[['Neighborhood', 'Italian Restaurant']]
indexColumn=['Neighborhood']

#dfTorontoCategory.reset_index(inplace=True)
dfTorontoCategory.set_index(indexColumn,inplace=True)
dfTorontoCategory['sum'] = dfTorontoCategory.groupby('Neighborhood')['Italian Restaurant'].sum()
dfTorontoCategory.reset_index(inplace=True)
toronto_part = dfTorontoCategory[['Neighborhood', 'Italian Restaurant','sum']]
toronto_part.drop(columns=['Italian Restaurant'],inplace=True)
toronto_part.rename(columns={'sum':'Italian Restaurant'}, inplace=True)
toronto_part.drop_duplicates(inplace=True)
toronto_part.sort_values(by=['Italian Restaurant'], inplace=True,ascending=False)
toronto_part

Explore the top 5 category in each venues. Then sort the venues in descending order and store information.


In [0]:
#dfTCategoryGroup.to_excel('DebugOutput.xls')
#from google.colab import files
#files.download('DebugOutput.xls') 
top_venue_num = 5
for hood in dfTCategoryGroup['Neighborhood']:
    #print("----"+hood+"----")
    temp = dfTCategoryGroup[dfTCategoryGroup['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venue_num))
    #print('\n')

Now for simple exploration we plot restaurant category. Alert: there is a generic category "Restaurant"

In [0]:
restaurantCat = dfTorontoCategory[
    dfTorontoCategory.columns[dfTorontoCategory.columns.str.contains("Restaurant", case=False)]
].sum()

#display a plot
restaurantCat.sort_values().plot(kind="barh", figsize=(12, 8))
plt.title("Total Number Of Different Restaurant Type", loc="left", fontsize=20)
plt.show()


Now for simple exploration we filter all the Venues data with category 'Italian Restaurant'


Now we aggregate only Top (x) Venue for Neighborhood


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#----------------------------
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Best Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Best Venue'.format(ind+1))

# create a dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfTCategoryGroup['Neighborhood']

for ind in np.arange(dfTCategoryGroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfTCategoryGroup.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## **Part.4 Predicting model.**
**Cluster Neighbourhoods with KMeans**

Now we use k_means to cluster the neighbourhood into x clusters.

## 4.1 KMeans Algorithm

In [0]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = dfTCategoryGroup.drop('Neighborhood', 1)
print(toronto_grouped_clustering)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [0]:
neighborhoods_venues_sorted.columns


In [0]:
#ONLY FOR DEBUG
from google.colab import files
#dfToronto.to_excel("Toronto.xls")
#files.download("Toronto.xls")

#toronto_part.to_excel("Toronto_part.xls")
#files.download("Toronto_part.xls")

indexColumn=['Neighborhood']
toronto_part.set_index(indexColumn,inplace=True)
toronto_part['IT Sum'] = toronto_part.groupby(["Neighborhood"])["Italian Restaurant"].count()
if 'Cluster Labels' not in dfTCategoryGroup:
    dfTCategoryGroup.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfToronto
#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_part, on='Neighborhood',rsuffix='n_')
#toronto_merged = toronto_merged.join(toronto_part.set_index('Neighborhood'), on='Neighborhood',rsuffix='n_')
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
toronto_merged.reset_index(drop=True, inplace=True)
toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()
# add clustering labels


Now we visualize the Clusters with Folium

In [0]:
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11, width='90%', height='70%')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Neighborhood'], 
                                  dfToronto['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# **Part.5 Examining the clusters**

Now Examining the Cluster from 0 to 5


Visualization of Cluster 0 . It is shown in red color in the map.

In [0]:
#Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

Visualization of Cluster 1 . It is shown in It is shown in purple color in the map.

In [0]:
 #Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

Visualization of Cluster 2

In [0]:
 #Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

Visualization of Cluster 3

In [0]:
 #Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

Visualization of Cluster 4

In [0]:
 #Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

Visualization of Cluster 5

In [0]:
 #Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5]